In [5]:
from django.db import models

import pandas as pd
import statsmodels.api as sm
import logging
import math

class CycleValidator:
    

    def __init__(self, data, mapDict):
        
        self.data = data
        self.mapDict = mapDict
        com_power = (self.data[self.mapDict['Commanded_Torque']] * self.data[self.mapDict['Commanded_Speed']] / 9.5488) / 1000
        self.data['Commanded_Power'] = com_power

        self.dataDict = {'Speed': [self.mapDict['Commanded_Speed'], self.mapDict['Engine_Speed']],
                         'Torque': [self.mapDict['Commanded_Torque'], self.mapDict['Engine_Torque']],
                         'Power': ["Commanded_Power", self.mapDict['Engine_Power']]}
        
        self._pre_regression_filter()
        self._regression()


        
    def _pre_regression_filter(self):

        curbidle = 600
        indeces = self.data[(self.data[self.mapDict['Commanded_Throttle']] == 0) & (self.data[self.mapDict['Commanded_Torque']] < 0)].index
        self.data = self.data.drop(indeces)
        indeces = self.data[(self.data[self.mapDict['Commanded_Throttle']] == 0) & (self.data[self.mapDict['Commanded_Speed']] == curbidle) &
                            ((self.data[self.mapDict['Commanded_Torque']] - (0.02 * self.data[self.mapDict['Commanded_Torque']].max())) < self.data[self.mapDict['Engine_Torque']] ) & 
                            ((self.data[self.mapDict['Commanded_Torque']] + (0.02 * self.data[self.mapDict['Commanded_Torque']].max())) > self.data[self.mapDict['Engine_Torque']] )].index

        self.data = self.data.drop(indeces)
        #'Torque': [self.mapDict['Commanded_Torque'], self.mapDict['Engine_Torque']],
        #indeces = self.data
        

    def _regression(self):
        
        self.reg_results = { 'Speed': { 'slope': " ", 'intercept': " ", 'standard_error': " ", 'rsquared': " " },
                             'Torque': { 'slope': " ", 'intercept': " ", 'standard_error': " ", 'rsquared': " " },
                             'Power': { 'slope': " ", 'intercept': " ", 'standard_error': " ", 'rsquared': " " }}

        for channel in self.dataDict.items():
           self._regression_util(channel[0], channel[1][0], channel[1][1])
 

In [173]:
from django.db import models

import pandas as pd
import statsmodels.api as sm
import logging
import math
import numpy as np

# Import Data
data = pd.read_csv("Test.csv", header=2,encoding="Windows-1252")
torque_demand = data.get("UDPi_TorqueDemand").convert_objects(convert_numeric=True)
torque_eng = data.get("C_TRQENG").convert_objects(convert_numeric=True)
speed_demand = data.get("UDPi_SpeedDemand").convert_objects(convert_numeric=True)
speed_eng = data.get("C_SPDDYN").convert_objects(convert_numeric=True)

In [146]:
##### Max Check #####
max_speed = np.nanmax(speed_demand)[2]
max_torque = np.nanmax(torque_demand)[2]
torque_test = torque_demand-0.02*max_torque

for i in range(2,len(torque_eng)):
    if torque_demand[i] == max_torque:
        print("hallo")
        #if speed_eng[i]>speed_demand[i]*0.98 & torque_eng[i] > torque_test[i]:
         #   print("hallo")
    

SyntaxError: unexpected EOF while parsing (<ipython-input-146-8a4adc0f0fbf>, line 11)

In [174]:
##### Min Check #####
speed_demand = speed_demand.drop(np.where(speed_demand == 0)[0])
speed_demand.index = range(0,len(speed_demand))
torque_demand = torque_demand.drop(np.where(torque_demand == 0)[0])
torque_demand.index = range(0,len(torque_demand))
#min_speed = np.nanmin(speed_demand)
#min_torque = np.nanmin(torque_demand)

for i in range(2,len(speed_demand)):
    if speed_demand[i] == min_speed[i] & torque_demand_new[i] == min_torque[i]:
        print("Hallo")
#for i in range(1,len(speed_demand):
 #              if min_speed[1]=

KeyError: 2

In [175]:
torque_demand

0            NaN
1       0.075000
2       9.673782
3     104.921028
4     262.283325
5     379.444153
6     434.977753
7     451.039703
8     457.357635
9     452.726105
10    427.311798
11    372.696198
12    273.561127
13    118.344315
14     24.419924
...
1131    15.0
1132    15.0
1133    15.0
1134    15.0
1135    15.0
1136    15.0
1137    15.0
1138    15.0
1139    15.0
1140    15.0
1141    15.0
1142    15.0
1143    15.0
1144    15.0
1145    13.5
Name: UDPi_TorqueDemand, Length: 1146, dtype: float64